<a href="https://colab.research.google.com/github/Stonepia/Scipy-Lec/blob/master/2_1_Advanced_Python_Constructs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.1 Advanced Python Constructs

## 2.1.1 Iterators, generator expressions and generators

### Iterators

In [0]:
nums =[1,2,3]
iter(nums)

In [0]:
nums.__iter__() # another way

In [0]:
nums.__reversed__()

In [0]:
it = iter(nums)
print(next(it))
print(next(it))

1
2


### Generator expressions

A second way in which iterator objects are created is through **generator expressions**, the basis for** list comprehensions**. To increase clarity, a generator expression must always be enclosed in parentheses or an expression. If round parentheses are used, then a generator iterator is created. If rectangular parentheses are used, the process is short-circuited and we get a `list`.

In [0]:
(i for i in nums) # generator

<generator object <genexpr> at 0x7f300228b780>

In [0]:
[i for i in nums] # list

[1, 2, 3]

In [0]:
list(i for i in nums)

[1, 2, 3]

The list comprehension syntax also extends to **dictionary and set comprehensions**. A `set` is created when the generator expression is enclosed in curly braces. A `dict` is created when the generator expression contains “pairs” of the form key:value:

In [0]:
{i for i in range(3)}

{0, 1, 2}

In [0]:
{i:i**2 for i in range(3)}

{0: 0, 1: 1, 2: 4}

### Generators

A third way to create iterator objects is to call a generator function.

When `next` is called, the function is executed until the first `yield`. Each encountered yield statement gives a value becomes the return value of `next`. After executing the ` yield ` statement, the execution of this function is suspended.



In [0]:
def f():
  yield 1
  yield 2
 
gen = f()


In [0]:
next(gen)

1

In [0]:
next(gen)

2

In [0]:
next(gen)

StopIteration: ignored

### Bidirectional Communication

When the generator resumes execution after a `yield` statement, the caller can call a method on the generator object to either pass a value **into** the generator, which then is returned by the `yield` statement, or a different method to inject an exception into the generator.

The first of the new methods is  [send(value)](https://docs.python.org/3/reference/expressions.html#generator.send()), which is similar to **next()**, but passes value into the generator to be used for the value of the `yield` expression. In fact, `g.next()` and `g.send(None)` are equivalent.

The second new method is [throw(type, value=None, traceback=None)](https://docs.python.org/3/reference/expressions.html#generator.throw), which is equivalent to:


```
raise type, value, traceback
```
at the point of the yield statement.

Unlike raise (which immediately raises an exception from the current execution point), throw() first resumes the generator, and only then raises the exception.

What happens when an exception is raised inside the generator? It can be either raised explicitly or when executing some statements or it can be injected at the point of a `yield` statement by means of the `throw()` method. In either case, such an exception propagates in the standard manner: it can be intercepted by an `except` or `finally` clause, or otherwise it causes the execution of the generator function to be aborted and propagates in the caller.

For completeness’ sake, it’s worth mentioning that generator iterators also have a [ close() ](https://docs.python.org/3/reference/expressions.html#generator.close)method, which can be used to force a generator that would otherwise be able to provide more values to finish immediately. It allows the generator **\_\_del__** method to destroy objects holding the state of generator.

In [0]:
import itertools

def g():
  print('--start--')
  for i in itertools.count():
    print('--yielding %i--' % i)
    try:
      ans = yield i
    except GeneratorExit:
      print('--closing--')
      raise
    except Exception as e:
      print('--yield raised %r --' %e)
    else:
      print('--yield returned %s --' %ans)
      

it = g()

In [0]:
next(it)

--start--
--yielding 0--


0

In [0]:
it.send(11)

--yield returned 11 --
--yielding 1--


1

In [0]:
it.throw(IndexError)

--yield raised IndexError() --
--yielding 2--


2

In [0]:
it.close()

--closing--


### Chaining generators

Let’s say we are writing a generator and we want to yield a number of values generated by a second generator, a subgenerator. If yielding of values is the only concern, this can be performed without much difficulty using a loop such as

In [0]:
subgen = some_other_generator()
for v in subgen:
    yield v

However, if the subgenerator is to interact properly with the caller in the case of calls to send(), throw() and close(), things become considerably more difficult. The yield statement has to be guarded by a try..except..finally structure similar to the one defined in the previous section to “debug” the generator function. Such code is provided in PEP 380#id13, here it suffices to say that new syntax to properly yield from a subgenerator is being introduced in Python 3.3:

In [0]:
yield from some_other_generator()

This behaves like the explicit loop above, repeatedly yielding values from some_other_generator until it is exhausted, but also forwards send, throw and close to the subgenerator.

## Decorators

### Decorators implemented as classes and as functions

The only requirement on decorators is that they can be called with a single argument.  This means that decorators can be implemented as normal functions, or as classes with a __call__ method, or in theory, even as lambda functions.





In [0]:
def simple_decorator(function):
  print("doing decoration")
  return function

@simple_decorator
def function():
  print("inside func")
  
function()

doing decoration
inside func


In [0]:
def decorator_with_arguments(arg):
  print("defining the decorator")
  def _decorator(function):
    # in this inner function, arg is available too
    print("doing decoration, %r" %arg)
    return function
  return _decorator

@decorator_with_arguments("abc")
def function():
  print("inside func")
  
function()

defining the decorator
doing decoration, 'abc'
inside func


The two trivial decorators above fall into the category of decorators which return the original function. If they were to return a new function, an extra level of nestedness would be required. In the worst case, three levels of nested functions.

In [0]:
def replacing_decorator_with_args(arg):
  print("defining the decorator")
  def _decorator(function):
    def _wrapper(*args, **kwargs):
      print("inside wrapper, %r, %r" % (args, kwargs))
      return function(*args, **kwargs)
    return _wrapper
  return _decorator

@replacing_decorator_with_args("abc")
def function(*args, **kwargs):
  print("inside function, %r, %r" % (args, kwargs))
  return 14

function(12,13)

defining the decorator
inside wrapper, (12, 13), {}
inside function, (12, 13), {}


14

Compared to decorators defined as functions, complex decorators defined as classes are simpler. When an object is created, the **\_\_init\_\_** method is only allowed to return **None**, and the type of the created object cannot be changed. This means that when a decorator is defined as a class, it doesn’t make much sense to use the argument-less form: the final decorated object would just be an instance of the decorating class, returned by the constructor call, which is not very useful. Therefore it’s enough to discuss class-based decorators where arguments are given in the decorator expression and the decorator **\_\_init\_\_** method is used for decorator construction.

In [0]:
class decorator_class(object):
  def __init__(self, arg):
    # This method is called in the decorator expression
    print("in decorator init, %s" % arg)
    self.arg = arg
  def __call__(self, function):
    # This method is called to do the job
    print("in decorator call, %s" % self.arg)
    return function
  
deco_instance = decorator_class('foo')

in decorator init, foo


In [0]:
@deco_instance
def function(*args, **kwargs):
  print(" in function, %s, %s" %(args, kwargs))
  

in decorator call, foo


In [0]:
function()

 in function, (), {}


In reality, it doesn’t make much sense to create a new class just to have a decorator which returns the original function. Objects are supposed to hold state, and such decorators are more useful when the decorator returns a new object.

In [0]:
class replacing_decorator_class(object):
  def __init__(self,arg):
    # This method is called in the decorator expression
    print("in decorator init, %s" %arg)
    self.arg = arg
  def __call__(self, function):
    # This method is called to do the job
    print("in decorator call, %s" %self.arg)
    self.function = function
    return self._wrapper
  def _wrapper(self, *args, **kwargs):
    print("in the wrapper, %s %s" %(args, kwargs))
    return self.function(*args, **kwargs)
  
deco_instance = replacing_decorator_class('foo')

in decorator init, foo


In [0]:
@deco_instance
def function(*args, **kwargs):
  "Function doc"
  print("in function, %s %s" %(args, kwargs))
  
function(11,12)

in decorator call, foo
in the wrapper, (11, 12) {}
in function, (11, 12) {}


In [0]:
print(function.__doc__)

None


A decorator like this can do pretty much anything, since it can modify the original function object and mangle the arguments, call the original function or not, and afterwards mangle the return value.

### Copying the docstring and other attributes of the original function

When a new function is returned by the decorator to replace the original function, an unfortunate consequence is that the original function name, the original docstring, the original argument list are lost. Those attributes of the original function can partially be “transplanted” to the new function by setting `__doc__` (the docstring), `__module__` and `__name__` (the full name of the function), and `__annotations__` (extra information about arguments and the return value of the function available in Python 3). This can be done automatically by using [functools.update_wrapper](https://docs.python.org/3/library/functools.html#functools.update_wrapper).

In [0]:
import functools
def replacing_decorator_with_args(arg):
  print(" defining the decorator")
  def _decorator(function):
    print("doing decoration, %r" %arg)
    def _wrapper(*args, **kwargs):
      print("inside the wrapper, %r, %r" %(args, kwargs))
      return function(*args,**kwargs)
    return functools.update_wrapper(_wrapper, function)
  return _decorator


@replacing_decorator_with_args("abc")
def function():
  "extensive documentation"
  print("inside function")
  return 14

 defining the decorator
doing decoration, 'abc'


In [0]:
print(function.__doc__)

extensive documentation


To sum things up, decorators should always use functools.update_wrapper or some other means of copying function attributes.

### Examples in the standard library

 - [classmethod](https://docs.python.org/3/library/functions.html#classmethod) causes a method to become a “class method”, which means that it can be invoked without creating an instance of the class. When a normal method is invoked, the interpreter inserts the instance object as the first positional parameter, `self`. When a class method is invoked, the class itself is given as the first parameter, often called `cls`.
 
 Class methods are still accessible through the class’ namespace, so they don’t pollute the module’s namespace. Class methods can be used to provide alternative constructors:

In [0]:
class Array(object):
  def __init__(self, data):
    self.data = data
    
  @classmethod
  def fromfile(cls, file):
    data = numpy.load(file)
    return cls(data)

- [staticmethod](https://docs.python.org/3/library/functions.html#staticmethod) is applied to methods to make them “static”, i.e. basically a normal function, but accessible through the class namespace. This can be useful when the function is only needed inside this class (its name would then be prefixed with _), or when we want the user to think of the method as connected to the class, despite an implementation which doesn’t require this.

[Example](https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner)

- [property](https://docs.python.org/3/library/functions.html#property) is the pythonic answer to the problem of getters and setters. A method decorated with property becomes a getter which is automatically called on attribute access.

In [0]:
class A(object):
  @property
  def a(self):
    " an important attribute"
    return "a value"
  
A.a

In [0]:
A().a

'a value'

In this example, `A.a` is an read-only attribute. It is also documented: `help(A)` includes the docstring for attribute a taken from the getter method. Defining `a` as a property allows it to be a calculated on the fly, and has the side effect of making it read-only, because no setter is defined.

To have a setter and a getter, two methods are required, obviously.

In [0]:
class Rectangle(object):
  def __init__(self, edge):
    self.edge=edge
  
  @property
  def area(self):
    """
    Compute area.
    """
    return self.edge**2
  
  @area.setter
  def area(self, area):
    self.edge = area**.5

The way that this works, is that the `property` decorator replaces the getter method with a property object. This object in turn has three methods, `getter`, `setter`, and `deleter`, which can be used as decorators. Their job is to set the getter, setter and deleter of the property object (stored as attributes `fget`, `fset`, and `fdel`). The getter can be set like in the example above, when creating the object. When defining the setter, we already have the property object under `area`, and we add the setter to it by using the `setter` method. All this happens when we are creating the class.

Afterwards, when an instance of the class has been created, the property object is special. When the interpreter executes attribute access, assignment, or deletion, the job is delegated to the methods of the property object.

To make everything crystal clear, let’s define a “debug” example:

In [0]:
class D(object):
  @property
  def a(self):
    print("getting 1")
    return 1
  @a.setter
  def a(self, value):
    print("setting %r" %value)
  @a.deleter
  def a(self):
    print("deleting")
    
print(D.a)
print(D.a.fget)
print(D.a.fset)
print(D.a.fdel)

d = D()
d.a

<function D.a at 0x7f7131c139d8>
<function D.a at 0x7f7131c13a60>
<function D.a at 0x7f7131c13ae8>
getting 1


1

In [0]:
d.a = 2

setting 2


In [0]:
d.a

getting 1


1

In [0]:
del d.a

deleting


### 2.1.2.6  A while-loop removing decorator

Let’s say we have function which returns a lists of things, and this list created by running a loop. If we don’t know how many objects will be needed, the standard way to do this is something like:

In [0]:
def find_answers():
    answers = []
    while True:
        ans = look_for_next_answer()
        if ans is None:
            break
        answers.append(ans)
    return answers

This is fine, as long as the body of the loop is fairly compact. Once it becomes more complicated, as often happens in real code, this becomes pretty unreadable. We could simplify this by using `yield` statements, but then the user would have to explicitly call `list(find_answers())`.

We can define a decorator which constructs the list for us:

In [0]:
import functools
def vectorized(generator_func):
    def wrapper(*args, **kwargs):
        return list(generator_func(*args, **kwargs))
    return functools.update_wrapper(wrapper, generator_func)

Our function then becomes:

In [0]:
@vectorized
def find_answers():
    while True:
        ans = look_for_next_answer()
        if ans is None:
            break
        yield ans

### 2.1.2.7 A plutin registration system

This is a class decorator which doesn’t modify the class, but just puts it in a global registry. It falls into the category of decorators returning the original object:

In [0]:
class WordProcessor(object):
  PLUGINS =[]
  def process(self, text):
    for plugin in self.PLUGINS:
      text = plugin().cleanup(text)
    return text
  
  @classmethod
  def plugin(cls, plugin):
    cls.PLUGINS.append(plugin)
    

@WordProcessor.plugin
class CleanMdashesExtension(object):
  def cleanup(self, text):
    return text.replace('&mdash;', u'\N{em dash}')

Here we use a decorator to decentralise the registration of plugins. We call our decorator with a noun, instead of a verb, because we use it to declare that our class is a plugin for WordProcessor. Method plugin simply appends the class to the list of plugins.

A word about the plugin itself: it replaces HTML entity for em-dash with a real Unicode em-dash character. It exploits the unicode literal notation to insert a character by using its name in the unicode database (“EM DASH”). If the Unicode character was inserted directly, it would be impossible to distinguish it from an en-dash in the source of a program.

## 2.1.3 Context Manager

A context manager is an object with `__enter__` and `__exit__` methods which can be used in the `with` statement:

In [0]:
with manager as var:
    do_something(var)

is in the simplest case equivalent to

In [0]:
var = manager.__enter__()
try:
    do_something(var)
finally:
    manager.__exit__()

In [0]:
class closing(object):
  def __init__(self, obj):
    self.obj = obj
  def __enter__(self):
    return self.obj
  def __exit__(self, *args):
    self.obj.close()
with closing(open('/tmp/file', 'w')) as f:
  f.write('the contents\n') 

Here we have made sure that the f.close() is called when the with block is exited. Since closing files is such a common operation, the support for this is already present in the file class. It has an __exit__ method which calls close and can be used as a context manager itself:

In [0]:
with open('/tmp/file', 'a') as f:
  f.write('more contents\n')  

### 2.1.3.1 Catching exceptions


When an exception is thrown in the with-block, it is passed as arguments to `__exit__`. Three arguments are used, the same as returned by [sys.exc_info()](https://docs.python.org/3/library/sys.html#sys.exc_info): type, value, traceback. When no exception is thrown, `None` is used for all three arguments. The context manager can “swallow” the exception by returning a true value from `__exit__`. Exceptions can be easily ignored, because if `__exit__` doesn’t use return and just falls of the end, `None` is returned, a false value, and therefore the exception is rethrown after` __exit__` is finished.

The ability to catch exceptions opens interesting possibilities. A classic example comes from unit-tests — we want to make sure that some code throws the right kind of exception:

In [0]:
class assert_raises(object):
    # based on pytest and unittest.TestCase
    def __init__(self, type):
        self.type = type
    def __enter__(self):
        pass
    def __exit__(self, type, value, traceback):
        if type is None:
            raise AssertionError('exception expected')
        if issubclass(type, self.type):
            return True # swallow the expected exception
        raise AssertionError('wrong exception type')

with assert_raises(KeyError):
    {}['foo']

### 2.1.3.2 Using generators to define context managers

When discussing generators, it was said that we prefer generators to iterators implemented as classes because they are shorter, sweeter, and the state is stored as local, not instance, variables. On the other hand, as described in Bidirectional communication, the flow of data between the generator and its caller can be bidirectional. This includes exceptions, which can be thrown into the generator. We would like to implement context managers as special generator functions. In fact, the generator protocol was designed to support this use case.

In [0]:
@contextlib.contextmanager
def some_generator(<arguments>):
    <setup>
    try:
        yield <value>
    finally:
        <cleanup>


The [contextlib.contextmanager](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager) helper takes a generator and turns it into a context manager. The generator has to obey some rules which are enforced by the wrapper function — most importantly it must `yield` **exactly once**. The part before the yield is executed from `__enter__`, the block of code protected by the context manager is executed when the generator is suspended in yield, and the rest is executed in` __exit__`. If an exception is thrown, the interpreter hands it to the wrapper through ` __exit__ `arguments, and the wrapper function then throws it at the point of the `yield` statement. Through the use of generators, the context manager is shorter and simpler.

Let’s rewrite the `closing` example as a generator:

In [0]:
@contextlib.contextmanager
def closing(obj):
    try:
        yield obj
    finally:
        obj.close()

Let’s rewrite the `assert_raises` example as a generator:

In [0]:
@contextlib.contextmanager
def assert_raises(type):
    try:
        yield
    except type:
        return
    except Exception as value:
        raise AssertionError('wrong exception type')
    else:
        raise AssertionError('exception expected')